# Training a Feedforward Neural Network with the MNIST dataset

This code is adapted from Michael Nielsen's online book [*Neural Networks and Deep Learning*](http://neuralnetworksanddeeplearning.com/index.html). There are a few conventions followed here that look slightly different from how we may understand neural networks. In particular, we usually think of passing data through a layer using a matrix multiplication. This code does this calculation element-wise, which is only done for computational efficiency. My goal is to present this code step-by-step, offering explanation where I can.

To run each cell, press `Shift` + `Enter` when the cell is selected.

## Loading the data

These two functions will be called to load in our MNIST image data. Currently, there are 50,000 images stored in `mnist.pkl.gz` in binary format.

In [14]:
# These libraries are used to unzip and read in the binary data.
import pickle
import gzip
import json
import random

# This is the standard numerical library for Python.
import numpy as np

def load_data():
    """ Return a tuple containing (training_data, validation_data,
    test_data). 
    
    In particular, `training_data` is a list containing 50,000
    2-tuples `(x, y)`.  
    `x` is a 784-dimensional numpy.ndarray
    containing the input image.  
    
    `y` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for `x`.
    
    `validation_data` and `test_data` are lists containing 10,000
    2-tuples `(x, y)`. In each case, `x` is a 784-dimensional
    numpy.ndarry containing the input image, and `y` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to `x`.
    
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code.
    """
    
    # Unzip and load in the binary data.
    with gzip.open('mnist.pkl.gz', 'rb') as binary_file:
        tr_d, va_d, te_d = pickle.load(binary_file, encoding="latin1")
    
    # Flatten each image into a vector.
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    
    # Convert the class variable into e_j.
    training_results = [vectorized_result(y) for y in tr_d[1]]
    
    # Pair the flattened images with their corresponding e_j.
    training_data = zip(training_inputs, training_results)
    
    # Repeat the above process for validation and test data.
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """ Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network.
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [15]:
# Here's where we actually load in the data.
training_data, validation_data, test_data = load_data()

## Defining the activation function and its derivative.

For this network, we'll use the sigmoid activation function defined by
$$
    \sigma(z) = \frac{1}{1 + e^{-z}},
$$
with the derivative given by $\sigma'(z) = \sigma(z)(1-\sigma(z))$.

We can use other activation functions, and I've included a definition for the rectified linear unit (ReLU) function here.

In [16]:
def sigmoid(z):
    """ The sigmoid function. """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """ Derivative of the sigmoid function. """
    return sigmoid(z)*(1-sigmoid(z))

def relu(z):
    """ The rectified linear unit function. """
    return np.max(0, z)

## Defining the cost function.

This code defines both the mean squared error and the cross-entropy cost functions. By default, our `Network` object will use the cross-entropy cost.

In [17]:
class QuadraticCost(object):
    # Calling these 'staticmethod's just allows them to be used outside of this class definition.
    @staticmethod
    def fn(a, y):
        """ Return the cost associated with an output `a` and desired output
        `y`.
        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """ Return the error delta from the output layer. """
        return (a-y) * sigmoid_prime(z)

class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """ Return the cost associated with an output `a` and desired output
        `y`.  
        
        Note that np.nan_to_num is used to ensure numerical
        stability. In particular, if both `a` and `y` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan. The np.nan_to_num ensures that that is converted
        to the correct value (0.0).
        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """ Return the error delta from the output layer.  
        
        Note that the parameter `z` is not used by the method. 
        It is included in the method's parameters in order to make the 
        interface consistent with the delta method for other cost classes.
        """
        return (a-y)

## Defining the `Network` object

If you're new to object-oriented programming, you can think of this step as declaring a custom data type. For example, instead of an `integer` or `string` class, we'll declare a `Network` class. This class will have **attributes** (variables defined within the class) and **methods** (functions defined within the class).

I've tried to add comments to make this easier to understand. The red comments at the beginning of each method are the **docstrings**.

In [18]:
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        """ Define the attributes for the Network class.
        
        The list `sizes` contains the number of neurons in the respective
        layers of the network. For example, if the list was [2, 3, 1]
        then it would be a three-layer network, with the first layer
        containing 2 neurons, the second layer 3 neurons, and the
        third layer 1 neuron. The biases and weights for the network
        are initialized randomly, using
        `self.default_weight_initializer` (see docstring for that
        method).
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost = cost

    def default_weight_initializer(self):
        """ Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 divided by the square root of the number of
        weights connecting to the same neuron. 
        
        Initialize the biases using a Gaussian distribution with mean 0 and 
        standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """ Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1. Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        This weight and bias initializer uses the same approach as in
        Chapter 1, and is included for purposes of comparison.  It
        will usually be better to use the default weight initializer
        instead.
        """
        
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a, activation = sigmoid):
        """ Return the output of the network if `a` is input. """
        
        for b, w in zip(self.biases, self.weights):
            a = activation(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False,
            early_stopping_n = 0):
        """ Train the neural network using mini-batch stochastic gradient
        descent.  The `training_data` is a list of tuples `(x, y)`
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter `lmbda`.  The method also accepts
        `evaluation_data`, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.
        """

        # early stopping functionality:
        best_accuracy = 1

        training_data = list(training_data)
        n = len(training_data)

        if evaluation_data:
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)

        # early stopping functionality:
        best_accuracy = 0
        no_accuracy_change = 0

        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, len(training_data))

            print("Epoch %s training complete" % j)

            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
                
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))
                
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
                
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))

            # Early stopping:
            if early_stopping_n > 0:
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    no_accuracy_change = 0
                    #print("Early-stopping: Best so far {}".format(best_accuracy))
                else:
                    no_accuracy_change += 1

                if (no_accuracy_change == early_stopping_n):
                    #print("Early-stopping: No accuracy change in last epochs: {}".format(early_stopping_n))
                    return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """ Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        `mini_batch` is a list of tuples `(x, y)`, `eta` is the
        learning rate, `lmbda` is the regularization parameter, and
        `n` is the total size of the training data set.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """ Return a tuple `(nabla_b, nabla_w)` representing the
        gradient for the cost function C_x.  `nabla_b` and
        `nabla_w` are layer-by-layer lists of numpy arrays, similar
        to `self.biases` and `self.weights`.
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        
        # TODO: Add in the option for the softmax function in the final layer.
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # Backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """ Return the number of inputs in `data` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag `convert` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results `y` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in the docstrings for `load_data`.
        """
        
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]

        result_accuracy = sum(int(x == y) for (x, y) in results)
        return result_accuracy

    def total_cost(self, data, lmbda, convert = False):
        """ Return the total cost for the data set `data``.  
        
        The flag `convert` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the `accuracy` method, above.
        """
        
        cost = 0.0
        
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
            
            # Add on the regularization cost.
            cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights) # '**' - to the power of.
            
        return cost

    def save(self, filename):
        """ Save the neural network to the file `filename`. """
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        
        with open(filename, "w") as f:
            json.dump(data, f)

## Training and testing the Neural Network

So far we haven't actually created our network, we've just set up the framework for doing so. We may now create an instance of a network and pass in our desired hyper-parameters.

As an example, I'll initialize a `Network` instance with 3 layers of sizes 784, 30, and 10. Since the data are 28x28 pixels, the input layer has 28x28 = 784 units (nodes). The last layer has 10 units because there are 10 possible classes. 

The second layer can have any number of units, and we can add any number of layers with any number of units between the input layer and the output layer. Adding in extra layers will increase the training time.

In [19]:
# Build our network.
net = Network([784, 30, 10], cost = CrossEntropyCost);

# Initialize the weights and biases.
net.default_weight_initializer()

Now we're ready to begin training. I'll specify 100 epochs, a mini-batch size of 10, the learning rate `eta` as 0.5, and the regularization parameter `lmbda` as 0.1.

When I execute the next cell, the training will begin, and takes my computer (Intel i7-2600 CPU @ 3.40GHz, 4 Cores, 8 logical processors) about 12 minutes.

In [20]:
net.SGD(training_data, 100, 10, eta = 0.5, lmbda = 0.1, evaluation_data = test_data, monitor_evaluation_accuracy = True)

Epoch 0 training complete
Accuracy on evaluation data: 9371 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 9452 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9580 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9468 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9572 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9575 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9515 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9588 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9584 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9606 / 10000
Epoch 10 training complete
Accuracy on evaluation data: 9618 / 10000
Epoch 11 training complete
Accuracy on evaluation data: 9617 / 10000
Epoch 12 training complete
Accuracy on evaluation data: 9609 / 10000
Epoch 13 training complete
Accuracy on evaluation data: 9617 / 10000
Epoch 14 training complete
Accuracy on evalu

From the output, we see that after 100 epochs we obtained 95.65\% accuracy on the evaluation dataset.

In [ ]:
# Now make it possible to test individual cases or a collection of external test data.